In [1661]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import random

import os
from obspy.core import read

from sklearn.model_selection import train_test_split

from spectrogram_to_array import spectrogram

main_path = os.path.abspath("")

### EVENTS

In [1662]:
#DATAFRAME WITH ALL EVENTS
all_events_file_path = os.path.join(main_path, 'earthquakes_filtered.txt') #all events
all_events = pd.read_csv(all_events_file_path, sep=',')

try:
    all_events.drop(columns=['Unnamed: 0'], inplace=True) #automatically created column (idk why)
except:
    pass

all_events.head()

,event_id,event_ID,year,month,day,hour,minute,second,lat,lng,depth,mag_ML,std_dev_ML,mag_MA,std_dev_MA,category
0,0,0,2007,1,1,2,41,13.28,-21.65559,-68.41471,121.33,2.345,0.020,2.394,0.029,0
1,1,1,2007,1,1,2,47,7.83,-20.54848,-69.05857,102.79,1.114,0.033,1.305,0.031,0
2,2,2,2007,1,1,3,50,29.15,-21.86299,-68.53639,110.95,2.779,0.031,2.917,0.031,0
3,3,3,2007,1,1,4,19,27.82,-20.29515,-69.13106,95.79,1.401,0.017,1.571,0.023,0
4,4,4,2007,1,1,5,40,2.58,-21.23847,-70.05151,34.64,1.995,0.022,2.222,0.018,0


In [1663]:
#DATAFRAME FROM THE EXISTING FILES IN geofon_waveforms FOLDER
dataset_size = 20000

file_list = os.listdir(os.path.join(main_path, "geofon_waveforms"))
file_list = [int(file[:-6]) for file in file_list] #remove the '.mseed' ending and convert to int to get event_id
file_list = random.sample(file_list, dataset_size) #select a random sample of N events from all the files

train_events, test_events = train_test_split(file_list, test_size=0.2, random_state=42) #split the dataset into the training and testing part

train_events = pd.DataFrame(data = train_events, columns = ['event_id'])
train_events = pd.merge(left = train_events, right = all_events, on='event_id', how= 'inner')

test_events = pd.DataFrame(data = test_events, columns = ['event_id'])
test_events = pd.merge(left = test_events, right = all_events, on='event_id', how= 'inner')

In [1664]:
#TRAIN EVENTS
train_events.head()

,event_id,event_ID,year,month,day,hour,minute,second,lat,lng,depth,mag_ML,std_dev_ML,mag_MA,std_dev_MA,category
0,10343,10606,2007,11,9,8,56,15.44,-20.35042,-69.42389,85.72,1.446,0.032,1.582,0.023,0
1,53999,55470,2011,2,5,9,26,23.41,-20.00575,-69.26369,93.99,1.684,0.029,1.843,0.033,0
2,35788,36686,2009,9,4,17,11,24.82,-21.24302,-68.74364,110.57,1.593,0.035,1.711,0.030,0
3,17637,18059,2008,4,5,12,30,16.06,-21.19642,-68.79114,112.45,2.259,0.024,2.348,0.023,0
4,6645,6815,2007,7,21,22,10,32.24,-22.00562,-68.53488,119.46,2.152,0.037,2.213,0.043,0


In [1665]:
#TEST EVENTS
test_events.head()

,event_id,event_ID,year,month,day,hour,minute,second,lat,lng,depth,mag_ML,std_dev_ML,mag_MA,std_dev_MA,category
0,57283,58855,2011,4,30,1,21,36.47,-21.72339,-68.33011,127.63,2.116,0.027,2.148,0.018,0
1,60876,62544,2011,7,18,22,26,56.80,-20.55929,-69.79889,34.78,0.963,0.016,1.214,0.023,0
2,28748,29446,2009,2,27,18,11,15.28,-22.04996,-70.35787,30.65,0.810,0.009,1.047,0.010,0
3,32006,32773,2009,5,21,22,25,45.20,-21.12401,-68.49275,133.13,2.962,0.015,2.990,0.015,0
4,33370,34183,2009,6,25,10,34,51.13,-21.61521,-68.61016,106.48,2.520,0.029,2.636,0.029,0


## CUSTOM DATASET

In [1666]:
class event_dataset(Dataset):
    def __init__(self, dataset_type: str, transform = None) -> None:

        if dataset_type not in ['train', 'test']:
            raise KeyError("dataset_type has to be one of the follwoing: 'train', 'test' ")

        if dataset_type == "train":
            self.dataframe = train_events
        elif dataset_type == "test":
            self.dataframe = test_events

        self.data_direcotry = "geofon_waveforms"
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        event_id, label = int(row['event_id']), int(row['category'])

        #WAVEFORM FETCH
        file_name = f"{event_id}.mseed"
        waveform = read(os.path.join(main_path, self.data_direcotry, file_name))

        #WAVEFORM PREP
        spec_data = [spectrogram(data=trace.data,
                                    samp_rate = 40.0,
                                    log = True,
                                    wlen = 2,
                                    per_lap = 0.5,
                                    dbscale = False)[0] for trace in waveform]
        
        spec_data = np.stack(spec_data, axis = 0, dtype=np.float32) #stack arrays such that we have 4801 arrays of len(3) - as if it is rgb
        
        #WRITE AS TENSORS
        label = torch.tensor(data= label, dtype= torch.int64)
        spec_data = torch.from_numpy(spec_data)

        #CREATE SAMPLE
        sample = {'label': label,
                  'data': spec_data}
        
        if self.transform:
            sample['data'] = self.transform(sample['data'])

        return sample


## MODEL ARCHITECTURE

In [1667]:
class Simple_CNN(nn.Module):
    def __init__(self):
        super(Simple_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 18, 1024)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 64 * 32 * 18)
        x = F.relu(self.fc1(x))
        
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

In [1668]:
class Simple_CNN_v2(nn.Module): #79 % with kernel size 10 at conv1
    def __init__(self):
        super(Simple_CNN_v2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=1) 
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(33728, 1024)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 33728)
        x = F.relu(self.fc1(x))
        
        x = torch.sigmoid(self.fc2(x))
        return x

## CNN HYPER PARAMETERS

In [1669]:
batch_size = 4
num_epochs = 4
learning_rate = 0.1
N_batch_print_stats = 100 #print stats for every 100 processed batches


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Simple_CNN().to(device = device)

loss_funciton = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(params=model.parameters(),
                              lr = learning_rate)

train_loader = DataLoader(dataset = event_dataset(dataset_type='train'),#, transform=nn.functional.normalize),
                          batch_size = batch_size,
                          shuffle=True,
                          num_workers= 0)

test_loader = torch.utils.data.DataLoader(dataset = event_dataset(dataset_type='test'),#, transform=nn.functional.normalize),
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers = 0)

## CNN TRAINING

In [1670]:
def train_one_epoch(N_batch_stats: int):
    model.train(True)
    running_loss = 0.0
    running_accuracy = 0.0

    for batch_idx, sample in enumerate(train_loader):
        #load data
        labels = sample['label'].to(device)
        data = sample['data'].to(device)

        #optimizer zero grad
        optimizer.zero_grad()

        #forward pass
        labels_logit = model(data).squeeze()
        labels_pred = torch.round(torch.sigmoid(labels_logit))

        correct = torch.sum(labels == labels_pred).item()
        running_accuracy += correct / batch_size

        #calculate loss / accuracy
        loss = loss_funciton(labels_pred, labels.float())
        running_loss += loss.item()

        #loss backwards
        loss.backward()
        
        #optimizer step
        optimizer.step()

        #data checking
        #print(f'True Labels: {labels.tolist()}; Predicted Labels: {[int(i) for i in labels_pred.tolist()]}')
        # print(f'Correct {correct}')
        #print('DIFFERENT VALUE PREDICITONS') if len(set(labels_pred.tolist())) != 1  else  None
        
        if batch_idx %  N_batch_stats == N_batch_stats - 1:
            avg_loss_across_batches = running_loss / N_batch_stats
            avg_acc_across_batches = (running_accuracy / N_batch_stats)*100

            print('Batch {0}, Loss: {1:.3f}, Accuracy: {2:.1f}%'.format(batch_idx+1,
                                                                        avg_loss_across_batches,
                                                                        avg_acc_across_batches))
            running_loss = 0.0
            running_accuracy = 0.0

            print()



## CNN TESTING

In [1671]:

def validate_one_epoch():
    model.train(False)
    running_loss = 0.0
    running_accuracy = 0.0

    for i, sample in enumerate(test_loader):
        #load data
        true_labels = sample['label'].to(device)
        inputs = sample['data'].to(device)

        with torch.no_grad():
            #forward pass
            labels_logit = model(inputs).squeeze()
            labels_pred = torch.round(torch.sigmoid(labels_logit))

            correct = torch.sum(true_labels == labels_pred).item()
            
            running_accuracy += correct / batch_size
            loss = loss_funciton(labels_pred, true_labels.float())
            running_loss += loss.item()
    
    avg_loss_across_batches = running_loss / len(test_loader)
    avg_acc_across_batches = (running_accuracy / len(test_loader)) *100

    print('Val Loss: {0:.3f}, Val Accuracy: {1:.1f}%'.format(avg_loss_across_batches,
                                                            avg_acc_across_batches))
    print('***************************************************')
    print()   
        

# CNN RUN

In [1672]:
for epoch_idx in range(num_epochs):
    print(f'Epoch: {epoch_idx + 1}\n')

    train_one_epoch(N_batch_stats= N_batch_print_stats)
    validate_one_epoch()

print('finished training')

PATH = './simple_cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch: 1

Batch 100, Loss: 0.860, Accuracy: 64.2%

Batch 200, Loss: 0.833, Accuracy: 69.0%

Batch 300, Loss: 0.836, Accuracy: 69.0%



KeyboardInterrupt: 

### CNN FEATURE TESTING

In [634]:
row = train_events.iloc[650]
event_id, label = int(row['event_id']), int(row['category'])
file_name = f"{event_id}.mseed"
waveform = read(os.path.join(main_path, "geofon_waveforms", file_name))

In [635]:
#WAVEFORM PREP
spec_data = [spectrogram(data=trace.data,
                            samp_rate = 40.0,
                            log = True,
                            wlen = 2,
                            per_lap = 0.5,
                            dbscale = False)[0] for trace in waveform]

spec_data = np.stack(spec_data, axis = 0, dtype=np.float32) #stack arrays such that we have 4801 arrays of len(3) - as if it is rgb

#WRITE AS TENSORS
label = torch.tensor(data= label, dtype= torch.int64)
spec_data = torch.from_numpy(spec_data)

#CREATE SAMPLE
sample = {'label': label,
            'data': spec_data}

waveform_n = sample['data']
waveform_n = F.normalize(input = waveform_n)

# MODEL SHAPE TESTING

In [1460]:
class Simple_CNN_v2(nn.Module): #79 % with kernel size 10 at conv1
    def __init__(self):
        super(Simple_CNN_v2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=1) 
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(33728, 1024)
        self.fc2 = nn.Linear(1024, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1, 33728)
        x = F.elu(self.fc1(x))
        
        x = torch.sigmoid(self.fc2(x))
        return x

CNN = Simple_CNN_v2()
a =  CNN.pool(CNN.conv1(waveform_n))
print(f'Shape after Conv1: {CNN.conv1(waveform_n).shape}, After MAx Pool: {a.shape}')

b = CNN.pool(CNN.conv2(a))
print(f'Shape after Conv2: {CNN.conv2(a).shape}, After MAx Pool: {b.shape}')

c = CNN.pool(CNN.conv3(b))
print(f'Shape after Conv3: {CNN.conv3(b).shape}, After MAx Pool: {c.shape}')

d = c.view(-1, 33728)
print(f'Shape after x.view: {d.shape}')

e = CNN.fc1(d)
print(f'Shape after fc1: {e.shape}')

f = CNN.fc2(e)
print(f'Shape after fc2: {f.shape}')

g = torch.sigmoid(f)
print(f'Final Shape: {g.shape}')

Shape after Conv1: torch.Size([16, 254, 147]), After MAx Pool: torch.Size([16, 127, 73])
Shape after Conv2: torch.Size([32, 125, 71]), After MAx Pool: torch.Size([32, 62, 35])
Shape after Conv3: torch.Size([64, 62, 35]), After MAx Pool: torch.Size([64, 31, 17])
Shape after x.view: torch.Size([1, 33728])
Shape after fc1: torch.Size([1, 1024])
Shape after fc2: torch.Size([1, 1])
Final Shape: torch.Size([1, 1])


In [944]:
class Simple_CNN(nn.Module):
    def __init__(self):
        super(Simple_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 18, 1024)
        self.fc2 = nn.Linear(1024, 1)  # Assuming binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 32 * 18)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x


CNN = Simple_CNN()
a =  CNN.pool(CNN.conv1(waveform_n))
print(f'Shape after Conv1: {CNN.conv1(waveform_n).shape}, After MAx Pool: {a.shape}')

b = CNN.pool(CNN.conv2(a))
print(f'Shape after Conv2: {CNN.conv2(a).shape}, After MAx Pool: {b.shape}')

c = CNN.pool(CNN.conv3(b))
print(f'Shape after Conv2: {CNN.conv3(b).shape}, After MAx Pool: {c.shape}')

d = c.view(-1, 64 * 32 * 18)
print(f'Shape after x.view: {d.shape}')

e = CNN.fc1(d)
print(f'Shape after fc1: {e.shape}')

f = CNN.fc2(e)
print(f'Shape after fc2: {f.shape}')

g = torch.sigmoid(f)
print(f'Final Shape: {g.shape}')

Shape after Conv1: torch.Size([16, 256, 149]), After MAx Pool: torch.Size([16, 128, 74])
Shape after Conv2: torch.Size([32, 128, 74]), After MAx Pool: torch.Size([32, 64, 37])
Shape after Conv2: torch.Size([64, 64, 37]), After MAx Pool: torch.Size([64, 32, 18])
Shape after x.view: torch.Size([1, 36864])
Shape after fc1: torch.Size([1, 1024])
Shape after fc2: torch.Size([1, 1])
Final Shape: torch.Size([1, 1])


# OLD MODELS

In [524]:
class seismic_CNN_v2(nn.Module):
    def __init__(self) -> None:
        super(seismic_CNN_v2, self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride = 2)

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels= 64, kernel_size= 3, stride=1, padding=1)

        self.fc1 = nn.Linear(in_features= 9216 , out_features=512)
        self.fc2 = nn.Linear(in_features= 512, out_features= 16)
        self.fc3 = nn.Linear(in_features=16, out_features=1)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)

        x = F.relu(self.conv2(x))
        x = self.max_pool(x)

        x = F.relu(self.conv3(x))
        x = self.max_pool(x)

        x = F.relu(self.conv4(x))
        x = self.max_pool(x)

        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))

        return x

CNN = seismic_CNN_v2()
a =  CNN.max_pool(CNN.conv1(waveform_n))
print(f'Shape after Conv1: {CNN.conv1(waveform_n).shape}, After MAx Pool: {a.shape}')

b = CNN.max_pool(CNN.conv2(a))
print(f'Shape after Conv2: {CNN.conv2(a).shape}, After MAx Pool: {b.shape}')

c = CNN.max_pool(CNN.conv3(b))
print(f'Shape after Conv3: {CNN.conv3(b).shape}, After MAx Pool: {c.shape}')

d = CNN.max_pool(CNN.conv4(c))
print(f'Shape after Conv4: {CNN.conv4(c).shape}, After MAx Pool: {d.shape}')

e = torch.flatten(d, 0)
print(f'Shape after flatten: {e.shape}')

f = CNN.fc1(e)
print(f'Shape after fc1: {e.shape}')

g = CNN.fc2(f)
print(f'Shape after fc2: {f.shape}')

h = CNN.fc3(g)
print(f'Shape after fc3: {h.shape}')

i = CNN.sigmoid(h)
print(f'Final Shape: {i.shape}')

Shape after Conv1: torch.Size([16, 256, 149]), After MAx Pool: torch.Size([16, 128, 74])
Shape after Conv2: torch.Size([32, 128, 74]), After MAx Pool: torch.Size([32, 64, 37])
Shape after Conv3: torch.Size([64, 64, 37]), After MAx Pool: torch.Size([64, 32, 18])
Shape after Conv4: torch.Size([64, 32, 18]), After MAx Pool: torch.Size([64, 16, 9])
Shape after flatten: torch.Size([9216])
Shape after fc1: torch.Size([9216])
Shape after fc2: torch.Size([512])
Shape after fc3: torch.Size([1])
Final Shape: torch.Size([1])


In [429]:
class kaelynn_CNN(nn.Module):
    def __init__(self, dropout_prob) -> None:
        super(kaelynn_CNN, self).__init__()

        self.max_pool = nn.MaxPool2d(kernel_size=2, stride = 2)

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
      
        self.fc1 = nn.Linear(in_features= 296672, out_features=1024)
        self.fc2 = nn.Linear(in_features= 1024, out_features= 1)

        self.drop = nn.Dropout(p = dropout_prob)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)

        x = self.drop(x)

        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))

        x = self.drop(x)

        x = self.sigmoid(self.fc2(x))

        return x
    
CNN = kaelynn_CNN(dropout_prob=dropout_prob)
a =  CNN.max_pool(CNN.conv1(waveform_n))
print(f'Shape after Conv1: {CNN.conv1(waveform_n).shape}, After MAx Pool: {a.shape}')

b = CNN.drop(a)
print(f'Shape after drop1: {b.shape}')

c = torch.flatten(b, 0)
print(f'Shape after flatten: {c.shape}')

d = CNN.fc1(c)
print(f'Shape after fc1: {d.shape}')

e = CNN.drop(d)
print(f'Shape after drop1: {e.shape}')

f = CNN.fc2(e)
print(f'Shape after fc2: {f.shape}')

g = CNN.sigmoid(f)
print(f'Final Shape: {g.shape}')

Shape after Conv1: torch.Size([32, 254, 147]), After MAx Pool: torch.Size([32, 127, 73])
Shape after drop1: torch.Size([32, 127, 73])
Shape after flatten: torch.Size([296672])
Shape after fc1: torch.Size([1024])
Shape after drop1: torch.Size([1024])
Shape after fc2: torch.Size([1])
Final Shape: torch.Size([1])


In [501]:
class kaelynn_CNN_v2(nn.Module):
    def __init__(self, dropout_prob) -> None:
        super(kaelynn_CNN_v2, self).__init__()

        self.max_pool = nn.MaxPool2d(kernel_size=2, stride = 2)

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels= 128, kernel_size = 3)
      
        self.fc1 = nn.Linear(in_features= 277760, out_features=1024)
        self.fc2 = nn.Linear(in_features= 1024, out_features= 1)

        self.drop = nn.Dropout(p = dropout_prob)

        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)

        x = self.drop(x)

        x = x.view(-1,1)

        x = F.relu(self.fc1(x))

        x = self.sigmoid(self.fc2(x))

        return x
    
CNN = kaelynn_CNN_v2(dropout_prob=dropout_prob)
a =  CNN.max_pool(CNN.conv1(waveform_n))
print(f'Shape after Conv1: {CNN.conv1(waveform_n).shape}, After MAx Pool: {a.shape}')

aa =  CNN.max_pool(CNN.conv2(a))
print(f'Shape after Conv2: {CNN.conv1(waveform_n).shape}, After MAx Pool: {aa.shape}')

b = CNN.drop(aa)
print(f'Shape after drop1: {b.shape}')

c = b.view(1, -1)
print(f'Shape after .view: {c.shape}')

d = CNN.fc1(c)
print(f'Shape after fc1: {d.shape}')

f = CNN.fc2(e)
print(f'Shape after fc2: {f.shape}')

g = CNN.sigmoid(f)
print(f'Final Shape: {g.shape}')

Shape after Conv1: torch.Size([64, 254, 147]), After MAx Pool: torch.Size([64, 127, 73])
Shape after Conv2: torch.Size([64, 254, 147]), After MAx Pool: torch.Size([128, 62, 35])
Shape after drop1: torch.Size([128, 62, 35])
Shape after .view: torch.Size([1, 277760])
Shape after fc1: torch.Size([1, 1024])
Shape after fc2: torch.Size([1])
Final Shape: torch.Size([1])


In [725]:
class seismic_CNN_v3(nn.Module):
    def __init__(self) -> None:
        super(seismic_CNN_v3, self).__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride = 2)

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels= 64, out_channels= 128, kernel_size=3, stride=1, padding=1)

        self.drop = nn.Dropout(p = dropout_prob)


        self.fc1 = nn.Linear(in_features= 73728 , out_features=512)
        self.fc2 = nn.Linear(in_features= 512, out_features= 1)
    
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)

        x = F.relu(self.conv3(x))
        x = self.max_pool(x)

        x = self.drop(x)
        x = x.view(-1)

        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))

        return x

CNN = seismic_CNN_v3()
a =  CNN.max_pool(CNN.conv1(waveform_n))
print(f'Shape after Conv1: {CNN.conv1(waveform_n).shape}, After MAx Pool: {a.shape}')

b = CNN.max_pool(CNN.conv2(a))
print(f'Shape after Conv2: {CNN.conv2(a).shape}, After MAx Pool: {b.shape}')

c = CNN.max_pool(CNN.conv3(b))
print(f'Shape after Conv3: {CNN.conv3(b).shape}, After MAx Pool: {c.shape}')

d = CNN.drop(c)
print(f'Shape after drop: {d.shape}')

e = d.view(-1)
print(f'Shape after .view: {e.shape}')

f = CNN.fc1(e)
print(f'Shape after fc1: {e.shape}')

g = CNN.fc2(f)
print(f'Shape after fc2: {f.shape}')

i = CNN.sigmoid(g)
print(f'Final Shape: {i.shape}')

Shape after Conv1: torch.Size([32, 256, 149]), After MAx Pool: torch.Size([32, 128, 74])
Shape after Conv2: torch.Size([64, 128, 74]), After MAx Pool: torch.Size([64, 64, 37])
Shape after Conv3: torch.Size([128, 64, 37]), After MAx Pool: torch.Size([128, 32, 18])
Shape after drop: torch.Size([128, 32, 18])
Shape after .view: torch.Size([73728])
Shape after fc1: torch.Size([73728])
Shape after fc2: torch.Size([512])
Final Shape: torch.Size([1])
